# clean yearly econ data for 2015 to 2018 

In [3]:
import geopandas as gpd
import matplotlib as plt
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from shapely.geometry import Point
6
%matplotlib inline

import seaborn as sns
#plt.style.use('seaborn')
plt.rcParams['figure.dpi'] = 160

In [4]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame

import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point 
%matplotlib inline

In [9]:
ecodata15=pd.read_csv('../Data/ProjectData/CCA_2015_RAW.csv')
ecodata16=pd.read_csv('../Data/ProjectData/CCA_2016_RAW.csv')
ecodata17=pd.read_csv('../Data/ProjectData/CCA_2017_RAW.csv')
ecodata18=pd.read_csv('../Data/ProjectData/CCA_2018_RAW.csv')

ecodata=pd.read_csv('../Data/ProjectData/Community_Data_Snapshots_2023.csv')

In [10]:
print(f'2022: {ecodata.shape}')
print(f'2015: {ecodata15.shape}')
print(f'2016: {ecodata16.shape}')
print(f'2017: {ecodata17.shape}')
print(f'2018: {ecodata18.shape}')

2022: (77, 259)
2015: (77, 132)
2016: (77, 156)
2017: (77, 156)
2018: (77, 227)


In [11]:
print(ecodata15) #CCA
print(ecodata16) #GEOG
print(ecodata17) #GEOG
print(ecodata18) #GEOG

    OBJECTID             CCA  2000_POP  2010_POP  POP_CHANGE  TOT_EMP_RES  \
0          1     Albany Park     57655     51542       -6113        16850   
1          2  Archer Heights     12644     13393         749         4620   
2          3   Armour Square     12032     13391        1359         4599   
3          4         Ashburn     39584     41081        1497        15541   
4          5  Auburn Gresham     55928     48743       -7185        13475   
..       ...             ...       ...       ...         ...          ...   
72        73       West Lawn     29235     33355        4120        10676   
73        74    West Pullman     36649     29651       -6998         9481   
74        75      West Ridge     73199     71942       -1257        23969   
75        76       West Town     87435     82236       -5199        39396   
76        77        Woodlawn     27086     23740       -3346         6873   

                   RES_NAICS1_TYPE RES_NAICS2_TYPE  \
0                    

In [12]:
temp_dup=np.intersect1d(ecodata15.columns.values,ecodata16.columns.values)
# temp_dup=np.intersect1d(ecodata16.columns.values,temp_dup)
temp_dup=np.intersect1d(ecodata17.columns.values,temp_dup)
temp_dup=np.intersect1d(ecodata18.columns.values,temp_dup)
len(temp_dup)

126

In [13]:
print(temp_dup)

['2000_POP' '2010_POP' 'A20_34' 'A35_49' 'A50_64' 'AG' 'ASIAN' 'AVG_VMT'
 'Agperc' 'BACH' 'BLACK' 'BR_0_1' 'BR_2' 'BR_3' 'BR_4' 'BR_5' 'CARPOOL'
 'COMM' 'COMM_OTHER' 'COMMperc' 'DROVE_AL' 'EMP' 'HA_40_70' 'HA_70_00'
 'HA_AFT2000' 'HA_BEF1940' 'HISP' 'HS' 'HU_2UN' 'HU_3_4UN' 'HU_GT_5UN'
 'HU_SNG_ATT' 'HU_SNG_DET' 'HU_TOT' 'HV_150_300K' 'HV_300_500K'
 'HV_GT_500K' 'HV_LT_150K' 'INC_100_150K' 'INC_25_50K' 'INC_50_75K'
 'INC_75_100K' 'INC_GT_150' 'INC_LT_25K' 'IND' 'INDperc' 'INST' 'INSTperc'
 'IN_LBFRC' 'MED_AGE' 'MED_HA' 'MED_HV' 'MED_ROOMS' 'MF' 'MIX' 'MIXperc'
 'Mfperc' 'NOT_IN_LBFRC' 'OBJECTID' 'OPEN' 'OPENperc' 'OTHER' 'OWN_OCC_HU'
 'POP_16OV' 'POP_25OV' 'POP_HH' 'RENT_OCC_HU' 'RES_CITY1_COUNT'
 'RES_CITY1_TYPE' 'RES_CITY2_COUNT' 'RES_CITY2_TYPE' 'RES_CITY3_COUNT'
 'RES_CITY3_TYPE' 'RES_CITY4_COUNT' 'RES_CITY4_TYPE' 'RES_CITY5_COUNT'
 'RES_CITY5_TYPE' 'RES_NAICS1_COUNT' 'RES_NAICS1_TYPE' 'RES_NAICS2_COUNT'
 'RES_NAICS2_TYPE' 'RES_NAICS3_COUNT' 'RES_NAICS3_TYPE' 'RES_NAICS4_COUNT'
 'R

In [14]:
col_races=["WHITE", "HISP", "BLACK", "ASIAN", "OTHER"]
col_edu=["LT_HS", "HS", "SOME_COLL", "ASSOC", "BACH", "GRAD_PROF", "UND5"]

col_ages=["A5_19", "A20_34", "A35_49", "A50_64", "A65_74", "A75_84", "OV85"]  ##ages
col_emp=[ "IN_LBFRC", "EMP", "UNEMP", "NOT_IN_LBFRC","MED_HV"]    ## employment status
col_inc=["INC_LT_25K", "INC_25_50K", "INC_50_75K", "INC_75_100K", "INC_100_150K",\
         "INC_GT_150", "MEDINC", "INCPERCAP"] ## Household Income
col_dev=["COMPUTER", "ONLY_SMARTPHONE", "NO_COMPUTER", "INTERNET", "BROADBAND", "NO_INTERNET"]   ## Household Computer and Internet Access
col_comm=["TRANSIT_LOW_PCT", "TRANSIT_MOD_PCT", "TRANSIT_HIGH_PCT"]  ## Transit Availability of Resident and Job Locations
col_walk=["WALKABLE_LOW_PCT", "WALKABLE_MOD_PCT", "WALKABLE_HIGH_PCT"]  ## Walkability of Resident and Job Locations
          

In [15]:
all_raw=[ecodata15,ecodata16,ecodata17,ecodata18]

In [16]:
for name,df in zip(['15','16','17','18'], all_raw):
#     print(df.shape)
    try:
        t=df[col_races]
        print(t.shape)
    except Exception as e:
        print(e)

(77, 5)
(77, 5)
(77, 5)
(77, 5)


In [17]:
col_edu1=['LT_HS', 'HS', 'SOME_COLL',
       'ASSOC', 'BACH', 'GRAD_PROF']
for name,df in zip(['15','16','17','18'], all_raw):
    try:
        t=df[col_edu]
        print(t.shape)
    except Exception as e:
        print(e)

"['LT_HS', 'SOME_COLL', 'ASSOC', 'GRAD_PROF', 'UND5'] not in index"
"['LT_HS', 'SOME_COLL', 'ASSOC', 'GRAD_PROF', 'UND5'] not in index"
"['LT_HS', 'SOME_COLL', 'ASSOC', 'GRAD_PROF', 'UND5'] not in index"
"['UND5'] not in index"


In [18]:
col_ages1=['UND19', 'A20_34',
       'A35_49', 'A50_64', 'A65_79', 'OV80', 'MED_AGE',]
for name,df in zip(['15','16','17','18'], all_raw):
    try:
        t=df[col_ages1]
        print(t.shape)
    except Exception as e:
        print(e)

(77, 7)
(77, 7)
(77, 7)
"['A65_79', 'OV80'] not in index"


In [19]:
for name,df in zip(['15','16','17','18'], all_raw):
    try:
        t=df[col_emp]
        print(t.shape)
    except Exception as e:
        print(e)

(77, 5)
(77, 5)
(77, 5)
(77, 5)


In [20]:
for name,df in zip(['15','16','17','18'], all_raw):
    try:
        t=df[col_inc]
        print(t.shape)
    except Exception as e:
        print(e)

"['MEDINC', 'INCPERCAP'] not in index"
"['INCPERCAP'] not in index"
"['INCPERCAP'] not in index"
"['INCPERCAP'] not in index"


In [21]:
for name,df in zip(['15','16','17','18'], all_raw):
    try:
        t=df[col_dev]
        print(t.shape)
    except Exception as e:
        print(e)

"None of [Index(['COMPUTER', 'ONLY_SMARTPHONE', 'NO_COMPUTER', 'INTERNET', 'BROADBAND',\n       'NO_INTERNET'],\n      dtype='object')] are in the [columns]"
"None of [Index(['COMPUTER', 'ONLY_SMARTPHONE', 'NO_COMPUTER', 'INTERNET', 'BROADBAND',\n       'NO_INTERNET'],\n      dtype='object')] are in the [columns]"
"None of [Index(['COMPUTER', 'ONLY_SMARTPHONE', 'NO_COMPUTER', 'INTERNET', 'BROADBAND',\n       'NO_INTERNET'],\n      dtype='object')] are in the [columns]"
"None of [Index(['COMPUTER', 'ONLY_SMARTPHONE', 'NO_COMPUTER', 'INTERNET', 'BROADBAND',\n       'NO_INTERNET'],\n      dtype='object')] are in the [columns]"


In [22]:
for name,df in zip(['15','16','17','18'], all_raw):
    try:
        t=df[col_comm]
        print(t.shape)
    except Exception as e:
        print(e)

"None of [Index(['TRANSIT_LOW_PCT', 'TRANSIT_MOD_PCT', 'TRANSIT_HIGH_PCT'], dtype='object')] are in the [columns]"
"None of [Index(['TRANSIT_LOW_PCT', 'TRANSIT_MOD_PCT', 'TRANSIT_HIGH_PCT'], dtype='object')] are in the [columns]"
"None of [Index(['TRANSIT_LOW_PCT', 'TRANSIT_MOD_PCT', 'TRANSIT_HIGH_PCT'], dtype='object')] are in the [columns]"
"None of [Index(['TRANSIT_LOW_PCT', 'TRANSIT_MOD_PCT', 'TRANSIT_HIGH_PCT'], dtype='object')] are in the [columns]"


In [23]:
for name,df in zip(['15','16','17','18'], all_raw):
    try:
        t=df[col_walk]
        print(t.shape)
    except Exception as e:
        print(e)

"None of [Index(['WALKABLE_LOW_PCT', 'WALKABLE_MOD_PCT', 'WALKABLE_HIGH_PCT'], dtype='object')] are in the [columns]"
"None of [Index(['WALKABLE_LOW_PCT', 'WALKABLE_MOD_PCT', 'WALKABLE_HIGH_PCT'], dtype='object')] are in the [columns]"
"None of [Index(['WALKABLE_LOW_PCT', 'WALKABLE_MOD_PCT', 'WALKABLE_HIGH_PCT'], dtype='object')] are in the [columns]"
"None of [Index(['WALKABLE_LOW_PCT', 'WALKABLE_MOD_PCT', 'WALKABLE_HIGH_PCT'], dtype='object')] are in the [columns]"


In [24]:
ecodata15.columns.values

array(['OBJECTID', 'CCA', '2000_POP', '2010_POP', 'POP_CHANGE',
       'TOT_EMP_RES', 'RES_NAICS1_TYPE', 'RES_NAICS2_TYPE',
       'RES_NAICS3_TYPE', 'RES_NAICS4_TYPE', 'RES_NAICS5_TYPE',
       'RES_NAICS1_COUNT', 'RES_NAICS2_COUNT', 'RES_NAICS3_COUNT',
       'RES_NAICS4_COUNT', 'RES_NAICS5_COUNT', 'TOT_EMP_WORK',
       'WORK_NAICS1_TYPE', 'WORK_NAICS2_TYPE', 'WORK_NAICS3_TYPE',
       'WORK_NAICS4_TYPE', 'WORK_NAICS5_TYPE', 'WORK_NAICS1_COUNT',
       'WORK_NAICS2_COUNT', 'WORK_NAICS3_COUNT', 'WORK_NAICS4_COUNT',
       'WORK_NAICS5_COUNT', 'RES_CITY1_TYPE', 'RES_CITY2_TYPE',
       'RES_CITY3_TYPE', 'RES_CITY4_TYPE', 'RES_CITY5_TYPE',
       'RES_CITY1_COUNT', 'RES_CITY2_COUNT', 'RES_CITY3_COUNT',
       'RES_CITY4_COUNT', 'RES_CITY5_COUNT', 'WORK_CITY1_TYPE',
       'WORK_CITY2_TYPE', 'WORK_CITY3_TYPE', 'WORK_CITY4_TYPE',
       'WORK_CITY5_TYPE', 'WORK_CITY1_COUNT', 'WORK_CITY2_COUNT',
       'WORK_CITY3_COUNT', 'WORK_CITY4_COUNT', 'WORK_CITY5_COUNT',
       'AVG_VMT', 'PARK_ACC

In [25]:
ecodata15['GEOG']=ecodata15['CCA']
ecodata15['MEDINC']=ecodata15['MED_INC']
ecodata18['A65_79']=ecodata18['A65_74']
ecodata18['OV80']=ecodata18['A75_84']+ecodata18['OV85']

In [26]:
temp_dup=np.intersect1d(ecodata15.columns.values,ecodata16.columns.values)
# temp_dup=np.intersect1d(ecodata16.columns.values,temp_dup)
temp_dup=np.intersect1d(ecodata17.columns.values,temp_dup)
temp_dup=np.intersect1d(ecodata18.columns.values,temp_dup)
len(temp_dup)
temp_dup

array(['2000_POP', '2010_POP', 'A20_34', 'A35_49', 'A50_64', 'A65_79',
       'AG', 'ASIAN', 'AVG_VMT', 'Agperc', 'BACH', 'BLACK', 'BR_0_1',
       'BR_2', 'BR_3', 'BR_4', 'BR_5', 'CARPOOL', 'COMM', 'COMM_OTHER',
       'COMMperc', 'DROVE_AL', 'EMP', 'GEOG', 'HA_40_70', 'HA_70_00',
       'HA_AFT2000', 'HA_BEF1940', 'HISP', 'HS', 'HU_2UN', 'HU_3_4UN',
       'HU_GT_5UN', 'HU_SNG_ATT', 'HU_SNG_DET', 'HU_TOT', 'HV_150_300K',
       'HV_300_500K', 'HV_GT_500K', 'HV_LT_150K', 'INC_100_150K',
       'INC_25_50K', 'INC_50_75K', 'INC_75_100K', 'INC_GT_150',
       'INC_LT_25K', 'IND', 'INDperc', 'INST', 'INSTperc', 'IN_LBFRC',
       'MEDINC', 'MED_AGE', 'MED_HA', 'MED_HV', 'MED_ROOMS', 'MF', 'MIX',
       'MIXperc', 'Mfperc', 'NOT_IN_LBFRC', 'OBJECTID', 'OPEN',
       'OPENperc', 'OTHER', 'OV80', 'OWN_OCC_HU', 'POP_16OV', 'POP_25OV',
       'POP_HH', 'RENT_OCC_HU', 'RES_CITY1_COUNT', 'RES_CITY1_TYPE',
       'RES_CITY2_COUNT', 'RES_CITY2_TYPE', 'RES_CITY3_COUNT',
       'RES_CITY3_TYPE', 'RE

In [35]:
all_selected_columns=col_races+col_ages1+col_emp+col_inc+['GEOG','TOT_POP','TOT_HH','MED_ROOMS']
all_selected_columns.remove('INCPERCAP')
len(all_selected_columns)

28

In [36]:
all_selected_columns=np.array(all_selected_columns)

In [37]:
all_selected_columns

array(['WHITE', 'HISP', 'BLACK', 'ASIAN', 'OTHER', 'UND19', 'A20_34',
       'A35_49', 'A50_64', 'A65_79', 'OV80', 'MED_AGE', 'IN_LBFRC', 'EMP',
       'UNEMP', 'NOT_IN_LBFRC', 'MED_HV', 'INC_LT_25K', 'INC_25_50K',
       'INC_50_75K', 'INC_75_100K', 'INC_100_150K', 'INC_GT_150',
       'MEDINC', 'GEOG', 'TOT_POP', 'TOT_HH', 'MED_ROOMS'], dtype='<U12')

In [38]:
ecodata15.columns

Index(['OBJECTID', 'CCA', '2000_POP', '2010_POP', 'POP_CHANGE', 'TOT_EMP_RES',
       'RES_NAICS1_TYPE', 'RES_NAICS2_TYPE', 'RES_NAICS3_TYPE',
       'RES_NAICS4_TYPE',
       ...
       'BR_3', 'BR_4', 'BR_5', 'HV_LT_150K', 'HV_150_300K', 'HV_300_500K',
       'HV_GT_500K', 'MED_HV', 'GEOG', 'MEDINC'],
      dtype='object', length=134)

In [39]:
ecodata15[all_selected_columns].to_csv('../Data/ProjectData/econ15_2.csv',index=False)
ecodata16[all_selected_columns].to_csv('../Data/ProjectData/econ16_2.csv',index=False)
ecodata17[all_selected_columns].to_csv('../Data/ProjectData/econ17_2.csv',index=False)
ecodata18[all_selected_columns].to_csv('../Data/ProjectData/econ18_2.csv',index=False)

In [79]:
ecodata15=pd.read_csv('./data/econ/econ15.csv')
ecodata16=pd.read_csv('./data/econ/econ16.csv')
ecodata17=pd.read_csv('./data/econ/econ17.csv')
ecodata18=pd.read_csv('./data/econ/econ18.csv')

In [41]:
pd.isnull(ecodata15).sum()

WHITE           0
HISP            0
BLACK           0
ASIAN           0
OTHER           0
UND19           0
A20_34          0
A35_49          0
A50_64          0
A65_79          0
OV80            0
MED_AGE         0
IN_LBFRC        0
EMP             0
UNEMP           0
NOT_IN_LBFRC    0
MED_HV          0
INC_LT_25K      0
INC_25_50K      0
INC_50_75K      0
INC_75_100K     0
INC_100_150K    0
INC_GT_150      0
MEDINC          0
GEOG            0
TOT_POP         0
TOT_HH          0
MED_ROOMS       0
dtype: int64

In [42]:
pd.isna(ecodata15).sum()

WHITE           0
HISP            0
BLACK           0
ASIAN           0
OTHER           0
UND19           0
A20_34          0
A35_49          0
A50_64          0
A65_79          0
OV80            0
MED_AGE         0
IN_LBFRC        0
EMP             0
UNEMP           0
NOT_IN_LBFRC    0
MED_HV          0
INC_LT_25K      0
INC_25_50K      0
INC_50_75K      0
INC_75_100K     0
INC_100_150K    0
INC_GT_150      0
MEDINC          0
GEOG            0
TOT_POP         0
TOT_HH          0
MED_ROOMS       0
dtype: int64

In [94]:
ecodata15.loc[ecodata15[ecodata15['GEOG']=='O\'Hare'].index,'GEOG']='ohare'

In [99]:
ecodata15.GEOG.values

array(['Albany Park', 'Archer Heights', 'Armour Square', 'Ashburn',
       'Auburn Gresham', 'Austin', 'Avalon Park', 'Avondale',
       'Belmont Cragin', 'Beverly', 'Bridgeport', 'Brighton Park',
       'Burnside', 'Calumet Heights', 'Chatham', 'Chicago Lawn',
       'Clearing', 'Douglas', 'Dunning', 'East Garfield Park',
       'East Side', 'Edgewater', 'Edison Park', 'Englewood',
       'Forest Glen', 'Fuller Park', 'Gage Park', 'Garfield Ridge',
       'Grand Boulevard', 'Greater Grand Crossing', 'Hegewisch',
       'Hermosa', 'Humboldt Park', 'Hyde Park', 'Irving Park',
       'Jefferson Park', 'Kenwood', 'Lake View', 'Lincoln Park',
       'Lincoln Square', 'Logan Square', 'Lower West Side',
       'McKinley Park', 'Montclare', 'Morgan Park', 'Mount Greenwood',
       'Near North Side', 'Near South Side', 'Near West Side', 'New City',
       'North Center', 'North Lawndale', 'North Park', 'Norwood Park',
       'ohare', 'Oakland', 'Portage Park', 'Pullman', 'Riverdale',
       'R

In [64]:
ecodata18[ecodata18['GEOG']=='Oakland']['GEOG']='Oakland1'

/tmp/ipykernel_3791260/1541620989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecodata18[ecodata18['GEOG']=='Oakland']['GEOG']='Oakland1'


In [100]:
ecodata15[ecodata15['GEOG']=='Loop'].index

Index([], dtype='int64')

In [101]:
ecodata15=pd.read_csv('./Data/ProjectData/econ15.csv')